In [2]:
import csv
import random
import copy
import pandas as pd
import numpy as np
import math
from sklearn.cross_validation import train_test_split
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score

In [3]:
pwd

u'/Users/jiny/Documents/DS Training'

In [3]:
filename = 'pima-indians-diabetes.csv'
data = pd.read_csv(filename,header = None, delimiter=' *, *', engine='python')

In [4]:
data.columns = ['TPreganency', 'PGC', 'DBP', 'TSFT','SI','BMI','DPF','Age','Result']

In [19]:
C = pd.DataFrame(np.corrcoef(data, rowvar=0))
C.columns = data.columns
print(C)
# np.corrcoef(data,rowvar=0)

# data.corrcoef()

   TPreganency       PGC       DBP      TSFT        SI       BMI       DPF  \
0     1.000000  0.129459  0.141282 -0.081672 -0.073535  0.017683 -0.033523   
1     0.129459  1.000000  0.152590  0.057328  0.331357  0.221071  0.137337   
2     0.141282  0.152590  1.000000  0.207371  0.088933  0.281805  0.041265   
3    -0.081672  0.057328  0.207371  1.000000  0.436783  0.392573  0.183928   
4    -0.073535  0.331357  0.088933  0.436783  1.000000  0.197859  0.185071   
5     0.017683  0.221071  0.281805  0.392573  0.197859  1.000000  0.140647   
6    -0.033523  0.137337  0.041265  0.183928  0.185071  0.140647  1.000000   
7     0.544341  0.263514  0.239528 -0.113970 -0.042163  0.036242  0.033561   
8     0.221898  0.466581  0.065068  0.074752  0.130548  0.292695  0.173844   

        Age    Result  
0  0.544341  0.221898  
1  0.263514  0.466581  
2  0.239528  0.065068  
3 -0.113970  0.074752  
4 -0.042163  0.130548  
5  0.036242  0.292695  
6  0.033561  0.173844  
7  1.000000  0.238356  
8  

In [9]:
randomdata.head()
#randomdata.describe()
#randomdata.Result.shape

,TPreganency,PGC,DBP,TSFT,SI,BMI,DPF,Age,Result
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
len(randomdata.columns)

9

In [11]:
Rawdata = randomdata.values[:,:8]
Result = randomdata.values[:,8]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
...     Rawdata, Result, test_size=0.4, random_state=0)

In [30]:
#Using GaussianNB to preduct:
clf = GaussianNB()
clf.fit(X_train, y_train)
target_pred = clf.predict(X_test)
accuracy_score(y_test, target_pred, normalize = True)

0.75

In [13]:
#Using my own method to predict:
#prior probability
trainingsize = len(y_train)
positive = y_train == 1
negative = y_train == 0
print('The total size of result in training is {0}, in which there are {1} ones and {2} zeros.'.format(len(y_train),
 len(y_train[positive]), len(y_train[negative]))
 )
priorP = len(y_train[positive])*1.0/trainingsize
priorN = len(y_train[negative])*1.0/trainingsize


The total size of result in training is 460, in which there are 165 ones and 295 zeros.


In [14]:
X_train_p = X_train[positive]
X_train_n = X_train[negative]
NormalDist_p = np.stack((X_train_p.mean(axis = 0),
X_train_p.var(axis = 0)))
NormalDist_n = np.stack((X_train_n.mean(axis = 0),
X_train_n.var(axis = 0)))

In [15]:
result_p = priorP*np.prod(np.exp(-np.square(X_train - NormalDist_p[0])/2/NormalDist_p[1])/np.sqrt(2*math.pi*NormalDist_p[1]),axis = 1)
result_n = priorN*np.prod(np.exp(-np.square(X_train - NormalDist_n[0])/2/NormalDist_n[1])/np.sqrt(2*math.pi*NormalDist_n[1]),axis = 1)

In [16]:
y_trainv = np.zeros(len(y_train))
y_trainv[result_p > result_n] = 1

In [17]:
acc_train = len(y_train[y_trainv == y_train])*1.0/len(y_train)

In [18]:
test_p = priorP*np.prod(np.exp(-np.square(X_test - NormalDist_p[0])/2/NormalDist_p[1])/np.sqrt(2*math.pi*NormalDist_p[1]),axis = 1)
test_n = priorN*np.prod(np.exp(-np.square(X_test - NormalDist_n[0])/2/NormalDist_n[1])/np.sqrt(2*math.pi*NormalDist_n[1]),axis = 1)

In [19]:
y_testv = np.zeros(len(y_test))
y_testv[test_p > test_n] = 1
acc_test = len(y_test[y_testv == y_test])*1.0/len(y_test)

In [21]:
acc_test
acc_train

0.7760869565217391